# Tratamento do SCI - Pessoa Juridica Padrão

In [1]:
# Imports
import pandas as pd
from numpy import nan

#Função pra pontuar CNPJ
def mask_cnpj(raw_cnpj):
    mask = '%s%s.%s%s%s.%s%s%s/%s%s%s%s-%s%s'
    return mask % tuple(raw_cnpj)

#Função pra pontuar CPF
def mask_cpf(raw_cpf):
    mask = '%s%s%s.%s%s%s.%s%s%s-%s%s'
    return mask % tuple(raw_cpf)

frase = 'Lista exportada com sucesso! =)'

## Definições de Importação:

In [2]:
# Necessário salvar o arquivo na pasta abaixo e alterar o nome da variavel \"arquivo\"
pasta_import = 'C:\\Projetos\\Python\SCI\\'
arquivo_import = 'enr-16082021-4522'

df = pd.read_csv(pasta_import + arquivo_import + '.csv', sep=';')

## Tratamento de Dados:

In [3]:
#Retira os decimais e transforma em string para iniciar o tratamento da coluna;
df['SOCIO_CPF'] = df['SOCIO_CPF'].apply(lambda x: str(x).replace('.0',''))

#Trata os CNPJs do dataframe para inserir os zeros na frente;
df['CNPJ'] = df['CNPJ'].astype(str).str.zfill(14)
df['SOCIO_CPF'] = df['SOCIO_CPF'].astype(str).str.zfill(11)

#Aplica a máscara de CPF e CNPJ
cnpjs_tratados = [mask_cnpj(cnpj) for cnpj in df['CNPJ']]
df['CNPJ1'] = cnpjs_tratados

cpf0_tratado = [mask_cpf(cpf) for cpf in df['SOCIO_CPF']]
df['SOCIO_CPF1'] = cpf0_tratado

#Seleciona só as colunas necessárias pro output e renomeia;
df_socios = df.filter(items=['CNPJ1','RAZAO_SOCIAL','SOCIO_CPF1','SOCIO_NOME'])
df_socios = df_socios.rename(columns={'CNPJ1':'CNPJ','RAZAO_SOCIAL':'RAZÃO SOCIAL','SOCIO_CPF1':'CPF DO SÓCIO','SOCIO_NOME':'NOME DO SÓCIO'})

#Ordena as colunas de A-Z e remove duplicatas;
df_socios.sort_values(by='RAZÃO SOCIAL', axis=0, inplace=True)
df_socios.duplicated(subset=['CNPJ'], keep='first')

#Remove a frase abaixo por nada
df_socios['CPF DO SÓCIO'].replace('000.000.00n-an',nan, inplace=True)

#Mostra a quantidade de CNPJs que não foram enriquecidos
qtd_nulos = df_socios['CPF DO SÓCIO'].isnull().sum()
nulos = 'Tivemos {} SÓCIOS que não foram enriquecidos e foram removidos.'.format(qtd_nulos)

#Remove valores que não foram enriquecidos
df_socios.dropna(subset=['CPF DO SÓCIO'], how='any', axis=0, inplace=True)

#Sócios em PROPERCASE
df_socios['NOME DO SÓCIO'] = df_socios['NOME DO SÓCIO'].str.title()

KeyError: 'SOCIO_CPF'

## Definições de Exportação:

In [ ]:
#Cria um excel writer usando xlsxwriter como engine;
writer = pd.ExcelWriter(pasta_import + 'Output ' + arquivo_import + '.xlsx', engine='xlsxwriter')

#Escreve os dataframes em diferentes sheets dentro do arquivo final;
df_socios.to_excel(writer, sheet_name='Sócios Pessoa Juridica - SCI', index=False)

#Fecha o arquivo e salva no diretório;
writer.save()

#Log
print(frase)
print(nulos)

Lista exportada com sucesso! =)
Tivemos 0 SÓCIOS que não foram enriquecidos e foram removidos.


##### Developed By: Gus (gustavo.lima@cortex-intelligence.com)  